In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [22]:
idx= pd.date_range(start="2023-01-01", periods=6, freq="D")
idx = pd.RangeIndex(start=0, stop=len(idx), step=1)

if isinstance(idx, pd.DatetimeIndex):
    freq = idx.freq
else:
    freq = idx.step

y_train_per_level = pd.DataFrame(
    {"y_train": np.arange(6), "_level_skforecast": [1, 2] * 3},
    index= idx
).groupby("_level_skforecast")

print(freq)
y_train_per_level

1


In [23]:
# NOTE: Interleaved Nan values were excluded fom y_train. They are restored
y_train_per_level = {key: group for key, group in y_train_per_level}
y_train_per_level

{1:    y_train  _level_skforecast
 0        0                  1
 2        2                  1
 4        4                  1,
 2:    y_train  _level_skforecast
 1        1                  2
 3        3                  2
 5        5                  2}

In [24]:
y_train_per_level[1].index

Index([0, 2, 4], dtype='int64')

In [25]:
import re
import pytest
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._utils import _predict_and_calculate_metrics_one_step_ahead_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import (
    series_wide_dt_item_sales,
    series_dict_range_item_sales,
    series_dict_dt_item_sales,
    exog_wide_range_item_sales,
    exog_wide_dt_item_sales,
    series_dict_nans,
    exog_dict_nans,
)

In [29]:
forecaster = ForecasterRecursiveMultiSeries(
                    regressor          = Ridge(random_state=678),
                    lags               = 3,
                    transformer_series = StandardScaler(),
                    transformer_exog   = StandardScaler(),
                    differentiation    = 1,
                    forecaster_id      = 'multiseries_transformer_diff'
                )

initial_train_size = 927
metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
levels = ['item_1', 'item_2', 'item_3']

(
    X_train,
    y_train,
    X_test,
    y_test,
    X_train_encoding,
    X_test_encoding
) = forecaster._train_test_split_one_step_ahead(
        series             = series_dict_range_item_sales,
        exog               = exog_wide_range_item_sales,
        initial_train_size = initial_train_size,
    )

results = _predict_and_calculate_metrics_one_step_ahead_multiseries(
    forecaster=forecaster,
    series=series_dict_range_item_sales,
    X_train = X_train,
    y_train = y_train,
    X_test = X_test,
    y_test = y_test,
    X_train_encoding = X_train_encoding,
    X_test_encoding = X_test_encoding,
    levels = levels,
    metrics = metrics,
    add_aggregated_metric = True
)

      y_train _level_skforecast
4   -1.762523            item_1
5   -0.106383            item_1
6   -0.223529            item_1
7   -0.181292            item_1
8   -0.024651            item_1
..        ...               ...
922  0.129919            item_3
923  1.037589            item_3
924 -0.561985            item_3
925 -0.671003            item_3
926  0.107253            item_3

[2769 rows x 2 columns]
y_train              0
_level_skforecast    0
dtype: int64
Index([  4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
       ...
       917, 918, 919, 920, 921, 922, 923, 924, 925, 926],
      dtype='int64', length=2769)


ValueError: Invalid frequency: 1